# This notebook has been archived now

In [1]:
import sys
import requests
from datetime import datetime, timedelta

In [2]:
API_HOST = 'https://access.redhat.com/hydra/rest/securitydata'

In [3]:
endpoint = '/cve.json'
params = 'per_page=1000,page=10'

In [4]:
full_query = API_HOST + endpoint + '?' + params

In [5]:
r = requests.get(full_query)

In [6]:
r.status_code

200

In [7]:
cves = r.json()

In [8]:
len(cves)

1000

In [9]:
cve_resources=[]
for cve in cves:
    cve_resources.append(cve['resource_url'])

In [10]:
len(cve_resources)

1000

In [11]:
from pandas import DataFrame
cve_names = []
package_names = []
fix_states = []

In [12]:
PYXIS_URL="https://catalog.redhat.com/api/containers/v1"

In [13]:
image_tags = []
grades = []

In [14]:
for cve_resource in cve_resources:
    cve_response = requests.get(cve_resource)
    security_data = cve_response.json()
    if 'package_state' in security_data:
        for package in security_data['package_state']:
            if (package['fix_state'] == 'Affected' and ("openshift" in package['package_name']) ):
                endpoint = PYXIS_URL+"/repositories/registry/registry.access.redhat.com/repository/"+package['package_name']+"/images?page_size=100"
                r = requests.get(endpoint)
                image_data = r.json()
                if len(image_data['data']) == 0:
                    continue                
                for data in image_data['data']:
                    if 'freshness_grades' not in data:
                        continue
                    for grade in data['freshness_grades']:
                        if ("C" in grade['grade'] or  "D" in grade['grade'] or "E" in grade['grade'] or "F" in grade['grade']):
                            cve_names.append(security_data['name'])
                            package_names.append(package['package_name'])
                            fix_states.append(package['fix_state'])                                
                            image_tags.append(data['image_id'])
                            grades.append(grade['grade'])                                
                            break
                        else:
                            continue
                    break
                        


In [15]:
len(cve_names) , len(package_names) , len(fix_states) , len(grades), len(image_tags)

(185, 185, 185, 185, 185)

In [16]:
# !pip install openpyxl

In [17]:
# security_data['name']+','+package['package_name']+ ','+package['fix_state'] + "\n"
df = DataFrame({'CVE': cve_names, 'package': package_names, 'Fix State': fix_states, 'tag': image_tags, 'grade': grades})
df.to_excel('openshift_component_with_tag.xlsx', sheet_name='sheet1', index=False)